In [43]:
import cv2
import requests
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub
import logging
import absl.logging

# Suppress TensorFlow warnings
import warnings
warnings.filterwarnings("ignore")

# Suppress TensorFlow-specific logging messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppresses INFO and WARNING messages (0 = all, 1 = info, 2 = warnings, 3 = errors)

# Correct method for TensorFlow 2.x to suppress deprecation warnings
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [44]:
# Suppress absl warnings
logging.getLogger("absl").setLevel(logging.ERROR)

# Load the pre-trained EfficientDet model
detector = hub.load("https://tfhub.dev/tensorflow/efficientdet/d2/1")

In [45]:
class_to_ingredient = {
    1: "tomato",
    2: "onion",
    3: "potato",
    4: "garlic",
    5: "ginger",
    6: "green chili",
    7: "coriander leaves (cilantro)",
    8: "mint leaves",
    9: "curry leaves",
    10: "spinach (palak)",
    11: "paneer",
    12: "cumin seeds",
    13: "mustard seeds",
    14: "turmeric powder",
    15: "red chili powder",
    16: "garam masala",
    17: "coriander powder",
    18: "fennel seeds",
    19: "fenugreek seeds",
    20: "cardamom",
    21: "cloves",
    22: "cinnamon",
    23: "bay leaves",
    24: "asafoetida (hing)",
    25: "ghee",
    26: "rice",
    27: "wheat flour",
    28: "biryani masala",
    29: "chicken",
    30: "mutton",
    31: "fish",
    32: "eggs",
    33: "chana dal",
    34: "toor dal",
    35: "moong dal",
    36: "masoor dal",
    37: "urad dal",
    38: "coconut",
    39: "coconut milk",
    40: "jaggery",
    41: "tamarind",
    42: "lemon",
    43: "yogurt (curd)",
    44: "buttermilk",
    45: "milk",
    46: "cream",
    47: "cashews",
    48: "almonds",
    49: "raisins",
    50: "bitter gourd (karela)",
    51: "brinjal (eggplant)",
    52: "cauliflower",
    53: "capsicum (bell pepper)",
    54: "ladyfinger (okra)",
    55: "cabbage",
    56: "carrot",
    57: "green peas",
    58: "methi leaves (fenugreek)",
    59: "mooli (radish)",
    60: "gourd (lauki)",
    61: "pumpkin",
    62: "kala chana (black chickpeas)",
    63: "rajma (kidney beans)",
    64: "black gram (urad dal whole)",
    65: "poha (flattened rice)",
    66: "sooji (semolina)",
    67: "besan (gram flour)",
    68: "sabudana (tapioca pearls)",
    69: "mustard oil",
    70: "vegetable oil",
    71: "saffron",
    72: "poppy seeds",
    73: "whole wheat",
    74: "puffed rice",
    75: "pickle",
    76: "papad",
    77: "jowar",
    78: "bajra",
    79: "amchur (dried mango powder)",
    80: "khoya",
    81: "makhana (fox nuts)",
    82: "tandoori masala",
    83: "chaat masala",
    84: "pav bhaji masala",
    85: "sambar powder",
    86: "rasam powder",
    87: "idli rice",
    88: "black salt",
    89: "kasuri methi (dried fenugreek leaves)",
    90: "imli (tamarind)",
    91: "soya chunks",
    92: "gond (edible gum)",
    93: "badam (almond)",
    94: "kaju (cashew)",
    95: "dahi (curd)"
}


In [46]:
def capture_image():
    cap = cv2.VideoCapture(0)  # 0 is usually the primary webcam

    while True:
        ret, frame = cap.read()
        if ret:
            cv2.imshow('Capture Image', frame)  # Display the frame

            # Press 's' to save the captured image
            if cv2.waitKey(1) & 0xFF == ord('s'):
                img_name = "captured_image.jpg"
                cv2.imwrite(img_name, frame)
                print(f"Image saved as {img_name}")
                break

    cap.release()
    cv2.destroyAllWindows()

    return img_name

In [47]:
def detect_ingredients(image_path):
    # Load and preprocess image
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_resized = tf.image.resize(image_rgb, (512, 512))
    image_resized = np.expand_dims(image_resized, axis=0)
    
    # Detect objects
    detections = detector(image_resized)

    # Extract detection classes and confidence scores
    class_ids = detections['detection_classes'][0].numpy().astype(int)
    scores = detections['detection_scores'][0].numpy()

    print("Class IDs:", class_ids)
    print("Scores:", scores)

    return class_ids, scores

In [48]:
def get_detected_ingredients(class_ids, scores, class_to_ingredient):
    threshold = 0.3  # Set the threshold for detection
    detected_ingredients = []

    # Iterate through the detections and map to ingredients
    for c, s in zip(class_ids, scores):
        if s > threshold:  # Filter by confidence threshold
            ingredient = class_to_ingredient.get(c, None)
            if ingredient:  # Only add valid ingredients
                detected_ingredients.append(ingredient)
    
    return detected_ingredients

In [49]:
def get_recipes(ingredients):
    api_key = '7e5a62c0114144bebabeefcdaacc68ac'  
    url = f"https://api.spoonacular.com/recipes/findByIngredients?ingredients={','.join(ingredients)}&number=5&apiKey={api_key}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        recipes = [recipe['title'] for recipe in data]
        return recipes
    else:
        print("Error fetching recipes from API:", response.status_code)
        return []

In [51]:
# Main sequence
image_path = capture_image()  # Capture image from webcam

# Detect ingredients from the captured image
class_ids, scores = detect_ingredients(image_path)

# Get the detected ingredients
detected_ingredients = get_detected_ingredients(class_ids, scores, class_to_ingredient)

# If ingredients are detected, fetch recipes based on them
if detected_ingredients:
    print("Detected Ingredients:", detected_ingredients)
    
    # Fetch recipes based on detected ingredients
    recipes = get_recipes(detected_ingredients)
    
    if recipes:
        print("\nHere are some recipe suggestions based on your ingredients:")
        for i, recipe in enumerate(recipes, 1):
            print(f"{i}. {recipe}")
    else:
        print("No recipes found.")
else:
    print("No ingredients detected in the image.")

Image saved as captured_image.jpg
Class IDs: [ 1  1 73 77 77 44  3 76 75 53 73 47  1 72 74 77 77  1 76 51 33 17 67  1
 27 31 52  1 60  1 86 80 76  1 37 85 73 75 77  1 10 27 65  1 27 13 77 84
 31  1 14 62 72 28 31 58 74 73 17 84 28 31 70 31 73 67 72 34 77 63  1  1
 79 51 31 77 16 78 47  3 51  1 40  1  1  1  1 31 59 75 73 77  1 84  3 62
 57 31 31 44]
Scores: [0.49884307 0.4810116  0.44055557 0.2543695  0.24701142 0.14124249
 0.13999473 0.13927421 0.13256076 0.13245562 0.13245557 0.12681921
 0.12662967 0.1175324  0.10852922 0.10237239 0.09997614 0.09733146
 0.0961562  0.09250254 0.09153971 0.08728524 0.0849869  0.08388931
 0.08110849 0.07866234 0.07602079 0.07588117 0.07476494 0.07373966
 0.07311001 0.07271508 0.07192331 0.07047309 0.07046369 0.07009879
 0.06974176 0.06918982 0.06834247 0.06834219 0.06644292 0.0625631
 0.06248494 0.06135231 0.06096532 0.06058609 0.05928613 0.05851008
 0.05838257 0.05800383 0.05793921 0.056968   0.05511654 0.05499629
 0.05278006 0.05230545 0.05209359 0.052